In [13]:
%load_ext autoreload
%autoreload 2

from utils import BPETokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
train_files = open("data/PY150K/python100k_train.txt", "r").read().split("\n")[:-1] # remove the last empty line
# train_texts = [open("data/PY150K/" + path, encoding='iso-8859-1').read() for path in train_files]

# tokenizer = BPETokenizer(initial_tokens="\n".join(train_texts)) # ensure that all unary tokens in our training data exist in our vocabulary
# tokenizer.fit("\n".join(train_texts[:1000]), 100) # fit the BPE on only the first 1000 files for computational reasons
# tokenizer.save("py150k_large")
tokenizer = BPETokenizer.load("py150k_large")

In [15]:
tokenizer.print_tokens(open("data/PY150K/" + train_files[10]).read())

# -*- coding: utf-8 -*-
from __future__ import unicode_literals

from django.db import models, migrations


class Migration(migrations.Migration):

    dependencies = [
        ('testreport', '0026_testresult_launch_item_id'),
    ]

    operations = [
        migrations.AddField(
            model_name='testplan',
            name='filter',
            field=models.TextField(default=b'', max_length=128, verbose_name='Started by filter', blank=True),
            preserve_default=True,
        ),
        migrations.AddField(
            model_name='testplan',
            name='main',
            field=models.BooleanField(default=False, verbose_name='Show in short statistic'),
            preserve_default=True,
        ),
    ]


In [16]:
from utils.dataset import PY150kDataset

ds = PY150kDataset("train", "py150k_large")
ds[1337]

tensor([ 82, 252, 212, 223, 219, 230,   4,  79, 236,  86,   9, 210,  78,  96,
         75,  88,  93,  23, 205, 208,  91,  79,  74, 240, 249,  50,  53,  88,
         80, 242,  91,   9,  82, 252, 212, 223,  50,  53,  88,  80, 242,  91,
          4,  79, 236,  86,   9, 210,  78,  96,  75,  88,  93,  23, 251, 252,
        206, 224,  93, 249, 292,  77, 283,  78, 249,  47, 212, 264,  93, 219,
         87,  80,   9,  82, 252, 212, 223,  47, 212, 264,  93, 219,  87,  80,
        231,  76,  85, 235, 215,  53,  88,  80,  47,  82, 229,  17,  50,  53,
         88,  80, 242,  91,  18,  35,   4,   3, 244,  11,   4,   3,  42,  87,
          9,  82, 252, 229, 230,  87,  93, 233,  82, 206,   9,  88, 222,  50,
         53,  88,  80, 242,  91,   9, 214, 233,   9, 203,  87,  77, 215, 230,
         92,  92,  74, 242, 215, 258,   9,  74,   9, 247, 229,  23,   4,   3,
        244,  11, 231,   3, 211, 222,  72,  72, 205,  82, 245,  72,  17, 203,
         85,  79, 207, 247, 229,  72, 284, 214,   9,  18,  35,  

One problem is that we need all sequences in a batch to be the same length, but there is a large difference in lengths

In [17]:
max(len(ds[i]) for i in range(100)), min(len(ds[i]) for i in range(100))

(80875, 23)

In [20]:
from utils.dataset import PY150kDataset
from torch.utils.data import DataLoader

def collate_fn(batch:list[torch.tensor]):
    min_length = min(len(x) for x in batch)
    # or pad to the maximum length
    return torch.stack([x[:min_length] for x in batch])

train_ds = PY150kDataset("train", "py150k_large")
train_dl = DataLoader(train_ds, batch_size=32, collate_fn=collate_fn)

For example purposes this will be a many-to-one encoder-decoder architecture. Our transformer atleast will probably be decoder only.

In [9]:
class PyLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x:torch.Tensor)->torch.Tensor:
        x = self.embed(x) # vocab to hidden
        x, (h_n, c_n) = self.rnn(x) # rnn also returns it's hidden/cell state
        x = self.linear(F.relu(x[:,-1])) # hidden to vocab
        return F.softmax(x)
    
model = PyLSTM(len(tokenizer), 128)
model(next(iter(train_dl))).shape

/tmp/ipykernel_157805/2546379325.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


torch.Size([32, 240])

https://wandb.ai/bjarnih/PyGPT

In [13]:
from tqdm import tqdm
import wandb

EPOCHS = 10
LR = 3e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

wandb.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": LR,
        "epochs": EPOCHS,
        "architecture": "many-to-one LSTM",
        "dataset": "small subset of PY150k",
    },
)

model = PyLSTM(len(tokenizer), 128).to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss().to(DEVICE)

for i in range(EPOCHS):
    for batch in tqdm(train_dl):
        x = batch[..., :-1].to(DEVICE).float()
        y = batch[..., -1].to(DEVICE) # keeps the unary dimension

        y_hat = model(x)

        loss = loss_fn(y, y_hat)

        optim.zero_grad()
        loss.backward()
        optim.step()

        wandb.log({"train_loss": loss.detach().numpy()})

  0%|          | 0/31 [00:00<?, ?it/s]ERROR:tornado.general:SEND Error: Host unreachable
